In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
import joblib


In [3]:


train_df = pd.read_csv(r'C:\Users\MSI\Videos\Downloads\archive\CICIOT23\train\train.csv')
test_df = pd.read_csv(r'C:\Users\MSI\Videos\Downloads\archive\CICIOT23\test\test.csv')
val_df = pd.read_csv(r'C:\Users\MSI\Videos\Downloads\archive\CICIOT23\validation\validation.csv')

# Print shapes
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Validation shape:", val_df.shape)



Train shape: (5491971, 47)
Test shape: (1176851, 47)
Validation shape: (1176851, 47)


In [4]:
train_normal = train_df[train_df['label'] == "BenignTraffic"].copy()
val_normal   = val_df[val_df['label'] == "BenignTraffic"].copy()


In [5]:
numeric_features = train_normal.select_dtypes(include=[np.number]).columns.tolist()

print("Number of numeric features:", len(numeric_features))


Number of numeric features: 46


In [6]:
scaler = MinMaxScaler()
scaler.fit(train_normal[numeric_features])

X_train = scaler.transform(train_normal[numeric_features])
X_val   = scaler.transform(val_normal[numeric_features])
X_test  = scaler.transform(test_df[numeric_features])


In [7]:
timesteps = X_train.shape[1]
n_features = 1 # Each of the 46 values is treated as a single feature over 46 timesteps

# Reshape data to be 3D [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0], timesteps, n_features)
X_val = X_val.reshape(X_val.shape[0], timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], timesteps, n_features)

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(timesteps, n_features), return_sequences=False),
    Dropout(0.2),
    RepeatVector(timesteps),
    LSTM(64, activation='tanh', return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(n_features))
])

model.compile(optimizer='adam', loss='mse')
model.summary()

c:\Users\MSI\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 46, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 46, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 46, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 46, 1)          │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,985 (195.25 KB)

 Trainable params: 49,985 (195.25 KB)

 Non-trainable params: 0 (0.00 B)